In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"
new_test = "./new_val_in/new_val_in"



# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

In [3]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
val_testset  = ArgoverseDataset(data_path=new_test)

In [78]:
batch_sz = 4
batch_sz_test = 1
def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    out = torch.FloatTensor(out)
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)


In [79]:
def my_collate1(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = torch.FloatTensor(inp)
    return inp

test_loader = DataLoader(val_testset, batch_size=batch_sz_test, shuffle=True, collate_fn=my_collate1)

In [5]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

        
for i_batch, sample_batch in enumerate(val_loader):
    inp, out = sample_batch
    show_sample_batch(sample_batch, agent_id)
    break

In [6]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv1d(2, 30, 2, 1)
        
        self.fc1 = nn.Linear(18, 2)
        

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        
        x = self.fc1(x)

        return x

In [7]:
from tqdm import tqdm_notebook as tqdm
import torch.nn as nn

def train(model, device, val_loader, optimizer, epoch, log_interval=10000):
    model.train()
    iterator = tqdm(val_loader, total=int(len(val_loader)))
    counter = 0
    mse_loss = nn.MSELoss()
    
    for batch_idx, (data, target) in enumerate(iterator):
        data, target = data.to(device), target.to(device)
        
        
        
        #Transpose data -> 4x19x60x4
        
        #data = torch.flatten(data, start_dim=2)
        
        data = data[0:4, 0, :, 0:2]
        data = torch.transpose(data,1,2)
        
        #reshape (conv1d) -> 4x19x240
        #conv2d -> 4x19x60x4 would work
        
        
        #output -> 4x1x30x4 -> 4x1x30x2 (only coords) -> reshape 4x60
        #input (only agent info & xy coords) -> 4x19x2
        
        #print("agent = ", agent.shape)
        
        target = target[0:4, 0, :, 0:2]
        target = torch.flatten(target, start_dim = 1)
        
        optimizer.zero_grad()
        output = model(data)
        output = torch.flatten(output, start_dim = 1)
        
        loss = mse_loss(output, target)
        
        loss.backward()
        optimizer.step()
        counter += 1
        iterator.set_postfix(loss=(loss.item()*data.size(0) / (counter * val_loader.batch_size)))
        

In [8]:
learning_rate = 0.01
momentum = 0.5
device = "cuda:0"
model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)
num_epoch = 2

print("Number of param: ", sum(p.numel()for p in model.parameters()))

for epoch in range(1, num_epoch + 1):
        train(model, device, val_loader, optimizer, epoch)
torch.save(model.state_dict(),"usingConv1D2epochs")

Number of param:  188


KeyboardInterrupt: 

In [17]:
# device = "cuda:0"
# model = CNN().to(device)
# model.load_state_dict(torch.load("usingConv1D"))
# model.eval()
# data = iter(val_loader)
# inputs, targets = next(data)

In [142]:
import csv
import pandas as pd
import numpy as np

device = "cuda:0"
model = CNN().to(device)
model.load_state_dict(torch.load("usingConv1D"))

filename = "output(1).csv"

def test(model, device, test_loader):
    model.eval()
    store = np.zeros(60)
    count = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            
            data = data[:, 0, :, 0:2]
            data = torch.transpose(data,1,2)
            
            #print("data = ", data.shape)
            
            output = model(data)
            output = output.cpu()         
            #output = torch.transpose(output,0,1)
            count+=1
            output_np = output.numpy()
            output_np = np.reshape(output_np,(1,60))
            #print(output_np)
            store = np.concatenate([store,output_np[0]],axis=-1)
            
            
#             output_df = pd.DataFrame(output_np)
#             output_df.to_csv('tmp.csv')
        print(store.shape)
        store = store[60:192060]
        store = np.reshape(store, (3200,60))
        print("store = ", store.shape)
        
        with open(filename, 'w',  newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerows(store)
            
            
              
test(model, device, test_loader)

(192060,)
store =  (3200, 60)
